# Construction of an index fund

An index fund is a selection of a small subset of the entire universe of stocks that we predict will closely track the index.

The model we now present attempts to cluster the stocks into groups that are “similar.”
Then one stock is chosen as the representative of each cluster.

The input data consists of parameters ρij that indicate the similarity of
each pair (i, j) of stocks in the market.

We have the following variables:
– yj is 1 if stock j is selected, 0 otherwise.
– xij is 1 if stock i is in the cluster represented by stock j, 0 otherwise.

The objective is to maximize the total similarity of all stocks to their representatives.

• We require that each stock be assigned to exactly one cluster and that the total number of clusters be q.


## Problem formulation

- $n$ is the number of stocks
- $ρ_{ij}$ indicates the similarity of each pair $(i, j)$ of stocks in the market.
- $y_{j}$  ∈ {0,1}, 1 if stock $j$ is selected, 0 otherwise. $j=\{1,2,..,n\}$
- $x_{ij}$ ∈ {0,1}, 1 if stock $i$ is in the cluster represented by stock $j$, 0 otherwise. $i=\{1,2,..,n\}$
- $q$ is the number of clusters. We require that each stock be assigned to exactly one cluster.

Then the formulation is the following
$$\begin{align}
\max \quad & \sum_{(i,j) \text{ ∈ [1,n]}} ρ_{ij} x_{ij} \\
\text{s.t.} \quad & \sum_{j=1}^n y_{j} = q \\
\ \quad & \sum_{j=1}^n x_{ij} = 1 && i\in \{1,2,..,n\} \\
\ \quad &  x_{ij} \leqslant y_{j} && i\in \{1,2,..,n\} && j\in \{1,2,..,n\} \\
\end{align}$$



### Importing libraries
- CVXPY : optimization problem solver

In [24]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import itertools

rcParams['figure.figsize'] = 16, 8

### Loading historical data

In [292]:
#Retrieving all the data from the .txt files
vinci_stocks = pd.read_csv('VINCI_2019-01-08.txt', sep="\t")
vinci_stocks = vinci_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])
sg_stocks = pd.read_csv('SOCIETEGENERALE_2019-01-08.txt', sep="\t")
sg_stocks = sg_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])
sanofi_stocks = pd.read_csv('SANOFI_2019-01-08.txt', sep="\t")
sanofi_stocks = sanofi_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])
bouyg_stocks = pd.read_csv('BOUYGUES_2019-01-08.txt', sep="\t")
bouyg_stocks = bouyg_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])
bnp_stocks = pd.read_csv('BNPPARIBASBRA_2019-01-08.txt', sep="\t")
bnp_stocks = bnp_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])
quantum_stocks = pd.read_csv('QUANTUMGENOMICS_2019-01-09.txt', sep="\t")
quantum_stocks = quantum_stocks.drop(columns=['ouv','haut','vol','devise','Unnamed: 7'])

In [293]:
vinci_stocks.rename({'clot':'VINCI'},axis=1,inplace=True)
bnp_stocks.rename({'clot':'BNP'},axis=1,inplace=True)
sg_stocks.rename({'clot':'SG'},axis=1,inplace=True)
sanofi_stocks.rename({'clot':'SANOFI'},axis=1,inplace=True)
bouyg_stocks.rename({'clot':'BOUYG'},axis=1,inplace=True)
quantum_stocks.rename({'clot':'QUANTUM'},axis=1,inplace=True)


In [294]:
vinci=np.array(vinci_stocks['VINCI'])
bouyg=np.array(bouyg_stocks['BOUYG'])
sanofi=np.array(sanofi_stocks['SANOFI'])
sg=np.array(sg_stocks['SG'])
bnp=np.array(bnp_stocks['BNP'])
quantum=np.array(quantum_stocks['QUANTUM'])

### Computing the correlation matrix

In [340]:
correlation_matrix=np.corrcoef([vinci,bouyg,sanofi,sg,bnp])
print('       [vinci    -    bouyg   -    sanofi   -    sg   -    bnp    ]')
correlation_matrix

       [vinci    -    bouyg   -    sanofi   -    sg   -    bnp    ]


array([[1.        , 0.69465691, 0.56456922, 0.42855814, 0.63032581],
       [0.69465691, 1.        , 0.17303234, 0.76440501, 0.76653873],
       [0.56456922, 0.17303234, 1.        , 0.2374724 , 0.42817254],
       [0.42855814, 0.76440501, 0.2374724 , 1.        , 0.92116859],
       [0.63032581, 0.76653873, 0.42817254, 0.92116859, 1.        ]])

In [341]:
correlation_vector=np.array([correlation_matrix[i,j] for i, j in itertools.product(range(4), range(4))])

### Problem framing

In [342]:
# Number of stocks
n=5
# Number of clusters
q=2
# x is a matrix of [n,n] boolean element
x = cvxpy.Variable((n,n), boolean=True)
#y is a vecotr of n boolean element
y = cvxpy.Variable((n), boolean=True)
#initilize a constraints list
constraints=[]
#sum of y vector is equal to the number of clusters
constraints.append(cp.sum(y)==q)
#inisialize a utility function
utility_function=0
for i in range(n):
    x_i_cst=0
    for j in range(n):
        constraints.append(x[i][j]<=y[j])
        x_i_cst+=x[i][j]
        utility_function+=x[i][j]*correlation_matrix[i][j]
    constraints.append(x_i_cst==1)


### Problem solving

In [343]:
index_fund_problem = cvxpy.Problem(cvxpy.Maximize(utility_function), constraints)
index_fund_problem.solve(verbose=True)
print("status:", index_fund_problem.status)
print("optimal value", index_fund_problem.value)

Iter	Lower Bound	Upper Bound	Gap

ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -3.244e+00  -3.884e+01  +4e+01  8e-03  9e-01  1e+00  5e-01    ---    ---    1  1  - |  -  - 
 1  -3.467e+00  -7.674e+00  +5e+00  9e-04  3e-01  1e-01  6e-02  0.8807  1e-03   0  0  0 |  0  0
 2  -4.119e+00  -5.072e+00  +1e+00  2e-04  6e-02  2e-02  1e-02  0.8441  8e-02   0  0  0 |  0  0
 3  -4.226e+00  -4.477e+00  +3e-01  5e-05  2e-02  6e-03  4e-03  0.7632  3e-02   0  0  0 |  0  0
 4  -4.288e+00  -4.332e+00  +5e-02  9e-06  3e-03  8e-04  6e-04  0.9085  9e-02   0  0  0 |  0  0
 5  -4.317e+00  -4.319e+00  +2e-03  3e-07  1e-04  3e-05  2e-05  0.9808  2e-02   0  0  0 |  0  0
 6  -4.318e+00  -4.318e+00  +2e-05  3e-09  1e-06  3e-07  3e-07  0.9890  1e-04   1  0  0 |  0  0
 7  -4.318e+00  -4.318e+00  +2e-07  4e-11  1e-08  3e-09  3e-09  0.9890  1e-04   1  0  0 |  0  0
 8  -4.318e+0

### Interpretating the solution 

In [344]:
stocks=['VINCI','BOUYG','SANOFI','SG','BNP']
print('Clusters representatives : ')
clusters_elements=[1 if y.value[i]>0.9 else 0 for i in range(n)]
print(clusters_elements)
print('Allocation matrix : ')
allocation=[[1 if x.value[i][j]>0.9 else 0 for j in range(n)] for i in range (n)]
print(allocation)
print('Allocation :')
for i in range(n):
    for j in range(n):
        if allocation[i][j]==1 :
            print(stocks[i],' is represented by ',stocks[j])


Clusters representatives : 
[0, 0, 1, 0, 1]
Allocation matrix : 
[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1]]
Allocation :
VINCI  is represented by  BNP
BOUYG  is represented by  BNP
SANOFI  is represented by  SANOFI
SG  is represented by  BNP
BNP  is represented by  BNP


# References

- https://coral.ie.lehigh.edu/~ted/files/ie447/lectures/Lecture15.pdf, Financial Optimization ISE 347/447, Dr. Ted Ralphs.